In [5]:
import time
import win32com.client
import pandas as pd


class Creon:
    def __init__(self):
        self.obj_CpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
        self.obj_CpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
        self.obj_StockChart = win32com.client.Dispatch('CpSysDib.StockChart')

    def creon_7400_주식차트조회(self, code, date_from, date_to):
        b_connected = self.obj_CpCybos.IsConnect
        if b_connected == 0:
            print("연결 실패")
            return None

        #list_field_key = [0, 1, 2, 3, 4, 5, 8]
        #list_field_name = ['date', 'time', 'open', 'high', 'low', 'close', 'volume']
        list_field_key = [0, 2, 3, 4, 5, 8]
        list_field_name = ['date', 'open', 'high', 'low', 'close', 'volume']
        dict_chart = {name: [] for name in list_field_name}

        self.obj_StockChart.SetInputValue(0, 'A'+code)
        self.obj_StockChart.SetInputValue(1, ord('1'))  # 0: 개수, 1: 기간
        self.obj_StockChart.SetInputValue(2, date_to)  # 종료일
        self.obj_StockChart.SetInputValue(3, date_from)  # 시작일
        self.obj_StockChart.SetInputValue(5, list_field_key)  # 필드
        self.obj_StockChart.SetInputValue(6, ord('D'))  # 'D', 'W', 'M', 'm', 'T'
        self.obj_StockChart.BlockRequest()

        status = self.obj_StockChart.GetDibStatus()
        msg = self.obj_StockChart.GetDibMsg1()
        print("통신상태: {} {}".format(status, msg))
        if status != 0:
            return None

        cnt = self.obj_StockChart.GetHeaderValue(3)  # 수신개수
        for i in range(cnt):
            dict_item = (
                {name: self.obj_StockChart.GetDataValue(pos, i) 
                for pos, name in zip(range(len(list_field_name)), list_field_name)}
            )
            for k, v in dict_item.items():
                dict_chart[k].append(v)

        print("차트: {} {}".format(cnt, dict_chart))
        return pd.DataFrame(dict_chart, columns=list_field_name)


if __name__ == '__main__':
    nu = '005930'
    creon = Creon()
    save_file = creon.creon_7400_주식차트조회(nu, 20150101, 20201231)
    print(save_file)
    save_file.to_csv('%s.csv'%(nu))
    

통신상태: 0 
차트: 1475 {'date': [20201230, 20201229, 20201228, 20201224, 20201223, 20201222, 20201221, 20201218, 20201217, 20201216, 20201215, 20201214, 20201211, 20201210, 20201209, 20201208, 20201207, 20201204, 20201203, 20201202, 20201201, 20201130, 20201127, 20201126, 20201125, 20201124, 20201123, 20201120, 20201119, 20201118, 20201117, 20201116, 20201113, 20201112, 20201111, 20201110, 20201109, 20201106, 20201105, 20201104, 20201103, 20201102, 20201030, 20201029, 20201028, 20201027, 20201026, 20201023, 20201022, 20201021, 20201020, 20201019, 20201016, 20201015, 20201014, 20201013, 20201012, 20201008, 20201007, 20201006, 20201005, 20200929, 20200928, 20200925, 20200924, 20200923, 20200922, 20200921, 20200918, 20200917, 20200916, 20200915, 20200914, 20200911, 20200910, 20200909, 20200908, 20200907, 20200904, 20200903, 20200902, 20200901, 20200831, 20200828, 20200827, 20200826, 20200825, 20200824, 20200821, 20200820, 20200819, 20200818, 20200814, 20200813, 20200812, 20200811, 20200810, 20

PermissionError: [Errno 13] Permission denied: '005930.csv'

In [9]:
k = save_file['close'].rolling(5).mean()

In [15]:
save_file[save_file['close'].isnull()]

,date,open,high,low,close,volume
